# Numerical integration: introduction

This notebook is based on Chapter 7 of 

<a id="thebook"></a>
> Süli, Endre and Mayers, David F. _An introduction to numerical analysis_. Cambridge University Press, Cambridge, 2003.
<https://doi.org/10.1017/CBO9780511801181> (ebook in [Helka](https://helka.helsinki.fi/permalink/358UOH_INST/1h3k2rg/alma9926836783506253))

We know how to evaluate some simple integrals of elementary functions by pen and paper but, in general, symbolic evaluation of integrals is a complicated problem, see for example the Wikipedia article of on the [Risch algorithm](https://en.wikipedia.org/wiki/Risch_algorithm). Here we consider numerical evaluation of integrals, a much simpler problem.

In [ ]:
import sympy as sp
from sympy.abc import x
from IPython.display import Math

def display_risch(f):
    int_tex = r'\int ' + sp.latex(f)  + ' dx'
    F = sp.integrate(f, x, risch=True)
    if not isinstance(F, sp.integrals.Integral):
        # The integral is elementary and we display the antiderivative
        display(Math(int_tex + '=' + sp.latex(F)))
    elif isinstance(F, sp.integrals.risch.NonElementaryIntegral):
        # The integral is proven to be nonelementary by the algorithm, 
        # meaning the integral cannot be represented using a combination of 
        # exponentials, logarithms, trig functions, powers, rational functions, 
        # algebraic functions, and function composition
        display(Math(int_tex + r'\,\text{ is not an elementary function}'))
    else:
        # The version implemented in SymPy only supports a small subset of 
        # the full Risch algorithm, no conclusion can be drawn from this case
        display(Math(int_tex + r'\,\text{ can not be handled by the algorithm}'))

In [ ]:
# Example from pp. 82-84 (Examples 2 and 3) of
#   Grozin, A. Introduction to Mathematica for Physicists. Springer, 2014.
# shows that a small change in the integrand makes it impossible to express 
# an integral by using elementary functions      
display_risch(sp.log(x)/x)
display_risch(sp.log(x)/(x + 1))
# Dubious example in the Wikipedia article on the Risch algorithm
display_risch(x / sp.sqrt(x**4 + 10*x**2 - 96*x - 72))

# Newton-Cotes formulae

Polynomials are easy to integrate, and the _Newton-Cotes formula_ of order $n = 1, 2, \dots$ is obtained by letting $p$ be the degree $n$ Lagrange interpolation polynomial of a function $f$ with equally-spaced points on an interval $[a,b]$, and approximating

$$
\int_a^b f(x) dx \approx \int_a^b p(x) dx.
$$

Recall that if $a = x_0 < x_1 < \dots < x_n = b$ are the equally spaced points, then

$$
p(x) = \sum_{k=0}^n f(x_k) L_k(x),
$$

where $L_k \in \mathbb P_n$ are the polynomials satisfying $L_k(x_i) = \delta_{ki}$.
Hence

$$
\int_a^b p(x) dx = \sum_{k=0}^n w_k f(x_k), \qquad w_k = \int_a^b L_k(x) dx.
$$

In [ ]:
import numpy as np
import scipy.interpolate as interp

def nc_weights(a, b, n):
    '''Compute the quadrature weights for Newton-Cotes formula of order n on [a, b]'''
    N = n+1
    xs = np.linspace(a, b, N)
    ws = np.zeros(N)
    for k in range(N):
        ys = np.zeros(N)
        ys[k] = 1
        l = interp.lagrange(xs, ys) # the polynomial L_k
        L = np.polyint(l)
        ws[k] = L(b) - L(a)
    return ws, xs

def nc_int(ws, xs, f):
    '''Integrate f using the quadrature weights ws and points xs'''
    return np.sum(ws * f(xs))

In [ ]:
a, b = 0, 1
def test_nc_int(ws, xs, expr):
    # Convert a symbolic expression with x as a free variable to a function 
    f = sp.lambdify(x, expr) 
    display(Math(r'\text{' + 
        f'Order n={np.size(ws)-1} gives {nc_int(ws, xs, f):.2f} ' +
        r' for }\int' + f'_{a}^{b}' + sp.latex(expr)  + 
        f' dx = {sp.integrate(expr, (x, a, b))}')) 

ws, xs = nc_weights(a, b, 1)
test_nc_int(ws, xs, 1)
test_nc_int(ws, xs, x)
test_nc_int(ws, xs, x**2)

In [ ]:
ws, xs = nc_weights(a, b, 2)
test_nc_int(ws, xs, 1)
test_nc_int(ws, xs, x)
test_nc_int(ws, xs, x**2)
test_nc_int(ws, xs, x**3)
test_nc_int(ws, xs, x**4)

It can be shown that the Newton-Cotes formula of order $n$ is _exact_, that is, it gives the exactly correct integral, for polynomials of degree $n$ if $n$ is odd and for polynomials of degree $n+1$ if $n$ is even.

## Theorem: error in Newton-Cotes
>
> Let $n=1,2,\dots$ and $f \in C^{n+1}(a,b)$. Let $w_i, x_i$, $i=0,\dots,n$, be the quadrature weights and points in the Newton-Cotes formula of order $n$ on $[a,b]$. Then
>
>\begin{align*}
&\left|\int_a^b f(x) dx - \sum_{k=0}^n w_k f(x_k) \right| 
\\&\quad\le 
\frac{\|f^{(n+1)}\|_\infty}{(n+1)!} \int_a^b (x-x_0)\dots(x-x_n) dx.
\end{align*}

For a proof, see Theorem 7.1 in [the book](#thebook). When the points $x_i$ are equally-spaced, the estimate is suboptimal for even $n$ since it does not take into account that the Newton-Cotes formula is exact for polynomials of order $n+1$ in this case. For a sharp estimate in the case $n=2$ with equal spacing, see Theorem 7.2 in [the book](#thebook).

In [ ]:
def nc_weights_sym(n):
    '''Compute symbolically the weights for Newton-Cotes formula of order n'''
    from sympy.abc import a, b, x
    ws = []
    xs = []
    h = (b - a)/n
    for k in range(n+1):
        xk = a + k*h
        xs.append(sp.simplify(xk))
        L = 1
        for i in range(n+1):
            if i != k:
                xi = a + i*h
                L *= (x - xi) / (xk -xi)
        ws.append(sp.simplify(sp.integrate(L, (x, a, b))))
    return ws, xs

def nc_formula(n):
    '''Compute symbolic expression for Newton-Cotes formula of order n'''
    w, x = nc_weights_sym(n)
    f = sp.Function('f')
    nc_form = 0
    for k in range(len(w)):
        nc_form += w[k] * f(x[k])
    return sp.simplify(nc_form)

In [ ]:
print('Trapezium rule')
display(Math(sp.latex(nc_formula(1))))
print("Simpson's rule")
display(Math(sp.latex(nc_formula(2))))

Recall that the Lagrange interpolation polynomial of 

$$
f(x) = \frac{1}{1+x^2}
$$

gives a poor approximation of $f$ on $[-5, 5]$ due to the Runge phenomenon. 

Thus the Newton-Cotes formulae are expected to work poorly for 

$$
\int_{-5}^5 f(x) dx.
$$

In [ ]:
def f(x):
    return 1 / (1 + x**2)
ns = range(2, 13, 2)
N = np.size(ns)
data = np.zeros(N)
for k in range(N):
    ws, xs = nc_weights(-5, 5, ns[k])
    data[k] = nc_int(ws, xs, f)

import pandas as pd
df = pd.DataFrame(data)
df.columns = ['$I_n$']
df.index = ns
df.index.name = 'n'
df.style.format('{:.5f}')

# Composite formulae

Instead of using a Newton-Cotes formula on a large interval, it is often better to divide the interval into small subintervals and apply the formula separately on each subinverval. We will consider only the trapezium rule. 

Let $a = x_0 < x_1 < \dots < x_n = b$ be equally spaced
and write $h = x_1 - x_0$. For $f \in C(a,b)$ there holds

$$
\int_a^b f(x) dx = \sum_{i=1}^n \int_{x_{i-1}}^{x_i} f(x) dx.
$$

We approximate using the trapezium rule

$$
\int_{x_{i-1}}^{x_i} f(x) dx \approx \frac{x_i - x_{i-1}}{2}(f(x_{i-1}) + f(x_i))
= 
\frac h 2 (f(x_{i-1}) + f(x_i)),
$$

and obtain the _composite trapezium rule_

$$
\int_a^b f(x) dx 
\approx 
h \left( \frac12 f(x_0) + f(x_1) + \dots + f(x_{n-1}) + \frac12 f(x_n) \right).
$$

## Theorem: error in composite trapezium rule
> Let $f \in C^2(a,b)$ and write $I_h f$ for the composite trapezium rule applied to $f$ with the spacing $h>0$. Then there is $C>0$, independent of $f$ and $h$, such that
>
>$$
\left|\int_a^b f dx - I_h f \right| \le \|(h\partial)^2 f\|_\infty.
$$

_Proof_. By the error in Newton-Cotes theorem,  

\begin{align*}
\left| \int_a^b f(x) dx - I_h f \right| 
\le 
\sum_{i=1}^n \left|\int_{x_{i-1}}^{x_i} f(x) dx - \frac h 2 (f(x_{i-1}) + f(x_i)) \right|
\le C n \|f''\|_\infty h^3.
\end{align*}

The claim follows from $h = (a-b)/n$.
$\blacksquare$

# On the integration sub-package of SciPy

The composite formulae are well-suited for approximation of integrals when the integrand is known only at some points $x_i$, $i=0,\dots,n$. The composite trapezium rule is implemented by [trapezoid](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.integrate.trapezoid.html) and its second order analogue, the composite Simpson's rule, by [simpson](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.integrate.simpson.html#scipy.integrate.simpson).

For highly regular grids $x_i$, $i=0,\dots,n$, there are methods that outperform the composite rules. If the samples are equally-spaced and the number of samples available is $2^k + 1$ for some integer $k=1,2,\dots$, then Romberg [romb](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.integrate.romb.html#scipy.integrate.romb) integration can be used to obtain high-precision estimates of the integral using the available samples. This method is described in Section 7.7 of [the book](#thebook).

If the integrand can be evaluated at any point on the region of integration, then [quad](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.integrate.quad.html) can be used. This calls the Fortran library [QUADPACK](https://en.wikipedia.org/wiki/QUADPACK). The main focus of this library is on automatic integration in which the algorithm attempts to perform an integration with an error no larger than that requested. The library uses an [adaptive quadrature](https://en.wikipedia.org/wiki/Adaptive_quadrature) that divides the region of integration into adaptively refined subintervals. Generally, adaptive algorithms are just as efficient as composite rules for smooth integrands, but they may work for non-smooth integrands for which traditional algorithms may fail. 

For more details on integration with SciPy see the [tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html).

In [ ]:
import scipy.integrate as integ
def f(x):
    return np.log(x)/(x+1)
a, b = 1, 2

xs = np.linspace(a, b)
ys = f(xs)
print(f'trapezoid gives {integ.trapezoid(ys, xs)}')
print(f'simpson gives   {integ.simpson(ys, xs)}')

xs, dx = np.linspace(a, b, 33, retstep=True)
ys = f(xs)
print(f'romb gives      {integ.romb(ys, dx)}')

# quad returns also an estimate of the absolute error in the result
print(f'quad gives     {integ.quad(f, a, b)}')

In [ ]:
def f(x):
    return 1/np.sqrt(x)
a, b = np.finfo(float).eps, 1

xs = np.linspace(a, b)
ys = f(xs)
print(f'trapezoid gives {integ.trapezoid(ys, xs)}')
print(f'simpson gives   {integ.simpson(ys, xs)}')

xs, dx = np.linspace(a, b, 33, retstep=True)
ys = f(xs)
print(f'romb gives      {integ.romb(ys, dx)}')

print(f'quad gives     {integ.quad(f, a, b)}')

print(f'correct value   2 (up to machine precision)')